# Higher Dimensional Datasets, version 10

## Load packages.

In [1]:
require(data.table)
require(deSolve)
require(magrittr)
require(randtoolbox)

Loading required package: data.table
Loading required package: deSolve
Loading required package: magrittr
Loading required package: randtoolbox
Loading required package: rngWELL
This is randtoolbox. For an overview, type 'help("randtoolbox")'.


## Function to generate simulations.

In [2]:
# Create a multivariate function with specified properties:
#   tmax: maximum time
#   multiplicities: number of correlations each parameter has
#   degrees: polynomial degree of each parameter
#   dimension: the dimension of the output
#   returns a multivariate function of the vector of parameters and time
makeGenerator <- function(multiplicities, degrees, dimension) {

    single <- function(degree) {
      x0 <- runif(1)
      z0 <- runif(1)
      function(x) {
          if (x < x0)
              0
          else
              z0 * (x - x0)^degree
      }
    }
  
    locations <- lapply(multiplicities, function(m) sample(1:dimension, m, replace=TRUE))
    functions <- lapply(degrees, single)
    
    start <- runif(dimension, -0.25, 0.75)
    coefs <- matrix(runif(dimension^2, -0.25, 0.75), dimension, dimension)    
    shift <- matrix(runif(dimension^2, -0.25, 0.75), dimension, dimension)
    
    function(x, ts) {
        z <- rep(0, dimension)
        for (i in 1:length(locations))
            for (j in locations[[i]])
                z[j] <- z[j] + functions[[i]](x[i])
        ode(start, ts, function(t, y, params) {list((coefs %*% y) * z * (1 - ((shift %*% y) * z)))})
    }
    
}


## Reproducible random numbers.

In [3]:
RNGkind("Mersenne-Twister", "Inversion", "Rejection")

In [4]:
set.seed(46)

## Create and save timeseries.

In [5]:
n <- 10000

In [6]:
ts <- seq(0, 3, 0.2)
ts

[1] 0.0 0.2 0.4 0.6 0.8 1.0 1.2 1.4 1.6 1.8 2.0 2.2 2.4 2.6 2.8 3.0

In [7]:
for (m in c(3, 20))
    for (k in c(3, 20))
        if (m != 3 || k != 3) {
            f.multiplicities <- sample(1:5, m, replace=TRUE)
            f.degrees <- sample(0:3, m, replace=TRUE)
            f <- makeGenerator(f.multiplicities, f.degrees, k)
            xs <- sobol(n, m)
            colnames(xs) <- paste("x", 1:m, sep="")
            rownames(xs) <- 1:n
            ys <- NULL
            for (i in 1:n) {
                x <- xs[i, ]
                if (m == 3)
                    y <- f(x, 10 * ts)
                else
                    y <- f(x, ts)
                ys <- rbind(ys, data.table(case=i, y))
            }
            xs <- data.table(case = 1:n, xs)
            colnames(ys) <- c("case", "time", paste("y", 1:k, sep=""))
            write.table(xs, file=paste("xs-", m, "x", k, "-v10.csv", sep=""), sep=",", quote=FALSE, row.names=FALSE)
            write.table(ys, file=paste("ys-", m, "x", k, "-v10.csv", sep=""), sep=",", quote=FALSE, row.names=FALSE)
        }

In [8]:
fread("ys-3x20-v10.csv") %>% summary

      case            time            y1                y2         
 Min.   :    1   Min.   : 0.0   Min.   :-0.2229   Min.   :0.02939  
 1st Qu.: 2501   1st Qu.: 7.5   1st Qu.:-0.2229   1st Qu.:0.02939  
 Median : 5000   Median :15.0   Median :-0.2229   Median :0.02939  
 Mean   : 5000   Mean   :15.0   Mean   :-0.2229   Mean   :0.02939  
 3rd Qu.: 7500   3rd Qu.:22.5   3rd Qu.:-0.2229   3rd Qu.:0.02939  
 Max.   :10000   Max.   :30.0   Max.   :-0.2229   Max.   :0.02939  
       y3               y4                y5               y6        
 Min.   :0.1046   Min.   :-0.1258   Min.   :0.1668   Min.   :0.1705  
 1st Qu.:0.1046   1st Qu.:-0.1258   1st Qu.:0.1668   1st Qu.:0.1705  
 Median :0.1046   Median :-0.1258   Median :0.1668   Median :0.1705  
 Mean   :0.1046   Mean   :-0.1258   Mean   :0.1668   Mean   :0.1709  
 3rd Qu.:0.1046   3rd Qu.:-0.1258   3rd Qu.:0.1668   3rd Qu.:0.1705  
 Max.   :0.1046   Max.   :-0.1258   Max.   :0.1668   Max.   :0.3275  
       y7                 y8      

In [9]:
fread("ys-20x3-v10.csv") %>% summary

      case            time            y1               y2        
 Min.   :    1   Min.   :0.00   Min.   :0.4993   Min.   :0.4195  
 1st Qu.: 2501   1st Qu.:0.75   1st Qu.:0.6134   1st Qu.:0.5818  
 Median : 5000   Median :1.50   Median :0.8127   Median :0.8661  
 Mean   : 5000   Mean   :1.50   Mean   :0.9544   Mean   :0.9165  
 3rd Qu.: 7500   3rd Qu.:2.25   3rd Qu.:1.1833   3rd Qu.:1.2168  
 Max.   :10000   Max.   :3.00   Max.   :2.9487   Max.   :1.9985  
       y3          
 Min.   :-1.00688  
 1st Qu.: 0.02767  
 Median : 0.10449  
 Mean   : 0.06336  
 3rd Qu.: 0.14834  
 Max.   : 0.52410  

In [10]:
fread("ys-20x20-v10.csv") %>% summary

      case            time            y1                 y2          
 Min.   :    1   Min.   :0.00   Min.   :-11.1354   Min.   :-3.99301  
 1st Qu.: 2501   1st Qu.:0.75   1st Qu.:  0.2260   1st Qu.:-0.91340  
 Median : 5000   Median :1.50   Median :  0.3583   Median :-0.41890  
 Mean   : 5000   Mean   :1.50   Mean   :  0.3625   Mean   :-0.48365  
 3rd Qu.: 7500   3rd Qu.:2.25   3rd Qu.:  0.4959   3rd Qu.: 0.06929  
 Max.   :10000   Max.   :3.00   Max.   :  2.0042   Max.   : 2.49500  
       y3               y4                y5                  y6          
 Min.   :0.5752   Min.   :-4.0103   Min.   :-0.372126   Min.   :-6.16373  
 1st Qu.:0.6035   1st Qu.: 0.4131   1st Qu.:-0.098232   1st Qu.:-0.57257  
 Median :0.6217   Median : 0.5349   Median : 0.003534   Median :-0.01752  
 Mean   :0.7131   Mean   : 0.5122   Mean   : 0.052012   Mean   :-0.34768  
 3rd Qu.:0.7603   3rd Qu.: 0.7352   3rd Qu.: 0.167168   3rd Qu.: 0.08266  
 Max.   :1.6130   Max.   :26.6816   Max.   : 0.809911   Max.